<a href="https://colab.research.google.com/github/xKDR/Shedding-light-on-the-Russia-Ukraine-war/blob/main/reproducible_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Russia - Ukraine Nightlights analysis (WIP)

#SETUP

##Get Assets

In [ ]:
using Pkg
Pkg.add(["GeoJSON", "NighttimeLights", "Rasters", "CSV", "Dates", "DataFrames", "GeoDataFrames"])

In [ ]:
# ;git clone https://github.com/xKDR/Shedding-light-on-the-Russia-Ukraine-war.git

In [ ]:
# ## Fetching the raster files
# ## This will take around 5 mins

# run(`apt-get install git-lfs -y`)
# run(`git lfs install`)
# cd("Shedding-light-on-the-Russia-Ukraine-war")
# run(`git lfs pull`)

In [ ]:
using GeoJSON
using GeoDataFrames
using DataFrames
using NighttimeLights
using Rasters
using CSV
using Dates

In [ ]:
# # Define path constants
# const FIGURES_DIRECTORY = "figures"
# const TABLES_DIRECTORY = "tables"
# const VECTOR_DIRECTORY = "data/vector"
# const TABULAR_DIRECTORY = "data/tabular"
# const RADIANCE_RASTER_DIRECTORY = "/content/Shedding-light-on-the-Russia-Ukraine-war/data/raster/rad_cropped"
# const CFOBS_RASTER_DIRECTORY = "/content/Shedding-light-on-the-Russia-Ukraine-war/data/raster/cf_cropped"

In [ ]:
# Define Local Paths
const FIGURES_DIRECTORY = "figures"
const TABLES_DIRECTORY = "tables"
const VECTOR_DIRECTORY = "data/vector"
const TABULAR_DIRECTORY = "data/tabular"
const RADIANCE_RASTER_DIRECTORY = "Shedding-light-on-the-Russia-Ukraine-war/data/raster/rad_cropped"
const CFOBS_RASTER_DIRECTORY = "Shedding-light-on-the-Russia-Ukraine-war/data/raster/cf_cropped"

In [ ]:
const START_DATE = Date(2019, 1)
const END_DATE = Date(2025, 1)

##Functions

In [ ]:
function safe_centre_of_mass(slice::AbstractArray, state_name::String)
    s = sum(skipmissing(slice))
    if ismissing(s) || isnan(s) || s == 0.0f0
        return [NaN, NaN]
    end
    try
        return NighttimeLights.centre_of_mass(slice)
    catch e
        if isa(e, InexactError)
            println("Warning: CoM calculation failed for '$state_name'. Returning [NaN, NaN].")
            return [NaN, NaN]
        else
            rethrow(e)
        end
    end
end

In [ ]:
function read_vector(filename)
# Use GeoJSON.read, which expects an IO
    geo = open(filename) do f
        GeoJSON.read(f)
    end

    # Now extract features
    features = geo.features

    # Build DataFrame with properties + geometry
    return DataFrame([merge(Dict(f.properties), Dict(:geometry => f.geometry)) for f in features])
end

In [ ]:
import Base.Filesystem: basename

"""
Extracts Date from filename.
Matches either `YYYY-MM-DD` or `YYYY-MM-DDTHH:MM:SS`.
"""
function file_to_date(f::AbstractString)
    b = basename(f)
    m = match(r"\d{4}-\d{2}-\d{2}(?:T\d{2}:\d{2}:\d{2})?", b)
    m === nothing && error("Cannot parse date from filename: $f")
    s = m.match
    if occursin('T', s)
        return Date(DateTime(s, dateformat"yyyy-mm-ddTHH:MM:SS"))
    else
        return Date(s, dateformat"yyyy-mm-dd")
    end
end


"""
Reads rasters between `start_date` and `end_date`
and returns cropped datacubes for radiance and cloud-free observations.
"""
function read_region_as_datacubes(region; start_date::Date, end_date::Date,
    radiance_path=RADIANCE_RASTER_DIRECTORY, cf_path=CFOBS_RASTER_DIRECTORY)

    # all .tif files
    rad_all = sort(joinpath.(radiance_path, filter(f -> endswith(f, ".tif"), readdir(radiance_path))))
    cf_all  = sort(joinpath.(cf_path, filter(f -> endswith(f, ".tif"), readdir(cf_path))))


    # restrict to date window
    rad_inwin = filter(f -> (d = file_to_date(f); start_date <= d <= end_date), rad_all)
    cf_inwin  = filter(f -> (d = file_to_date(f); start_date <= d <= end_date), cf_all)

    # sort by extracted date
    rad_inwin = rad_inwin[sortperm(file_to_date.(rad_inwin))]
    cf_inwin  = cf_inwin[sortperm(file_to_date.(cf_inwin))]

    # build raster lists
# build raster lists, now fully in memory
    rad_rasters = [Rasters.read(crop(Raster(f; lazy=true), to=region)) for f in rad_inwin]
    cf_rasters  = [Rasters.read(crop(Raster(f; lazy=true), to=region)) for f in cf_inwin]


    # raster series with actual dates
    rad_series = RasterSeries(rad_rasters, Ti(file_to_date.(rad_inwin)))
    cf_series  = RasterSeries(cf_rasters,  Ti(file_to_date.(cf_inwin)))

    # convert to datacubes
    rad_datacube = Rasters.combine(rad_series, Ti)
    cf_datacube  = Rasters.combine(cf_series, Ti)
    return rad_datacube, cf_datacube
end


In [ ]:
function process_state(state_row::DataFrameRow)
    state_name = state_row.NAME_1
    state_geometry = state_row.geometry
    @show state_name
    @show state_geometry
    radiance, cfobs = read_region_as_datacubes(state_geometry;
        start_date=START_DATE, end_date=END_DATE,
        radiance_path=RADIANCE_RASTER_DIRECTORY,
        cf_path=CFOBS_RASTER_DIRECTORY)


    print("file read correctly")

    radiance = clean_complete(radiance, cfobs; bgthreshold=4)
    cfobs = nothing # Release memory

    radiance = Raster(Float32.(radiance); missingval=missing)
    radiance[radiance .> 100.0f0] .= 0.0f0 # Cap high radiance values
    radiance = mask(radiance; with=state_geometry)

    dates = Date.(dims(radiance, Ti).val)
    num_timesteps = length(dates)

    if num_timesteps == 0
        println("Warning: No valid data for '$state_name' after cleaning.")
        return DataFrame()
    end

    agg_radiance = [sum(skipmissing(view(radiance, Ti(i)))) for i in 1:num_timesteps]
    com = [safe_centre_of_mass(view(radiance, Ti(i)), state_name) for i in 1:num_timesteps]

    return DataFrame(; dates, aggregate_radiance=agg_radiance, com, state_name)
end

#ANALYSIS

##Russia

In [ ]:
russia_state_split = read_vector("Shedding-light-on-the-Russia-Ukraine-war/data/vector/russia_processing.geojson")

In [ ]:
# russia_state_split = read_vector("/content/Shedding-light-on-the-Russia-Ukraine-war/data/vector/russia_processing.geojson")

In [ ]:
radiance, cfobs = read_region_as_datacubes(
    russia_state_split.geometry[1];
    start_date=START_DATE,
    end_date=END_DATE,
    radiance_path=RADIANCE_RASTER_DIRECTORY,
    cf_path=CFOBS_RASTER_DIRECTORY
)

In [ ]:
radiance, cfobs = read_region_as_datacubes(russia_state_split[1, :].geometry;
    start_date=START_DATE, end_date=END_DATE,
    radiance_path=RADIANCE_RASTER_DIRECTORY,
    cf_path=CFOBS_RASTER_DIRECTORY)

In [ ]:
russia_agg_rad = DataFrame()

for i in 1:nrow(russia_state_split)
    state_row = russia_state_split[i, :]
    state_name = state_row.NAME_1
    println("--- Processing state $i/$(nrow(russia_state_split)): $state_name ---")

    try
        state_df = process_state(state_row)
        if !isempty(state_df)
            append!(russia_agg_rad, state_df; cols=:union)
        end
    catch e
        println("ERROR: Failed to process state '$state_name'. Skipping. Error: $e")
    end
    GC.gc()
end

println("--- Processing Complete ---")

# Final aggregation
if !isempty(russia_agg_rad)
    println("Aggregating results...")
    grouped = groupby(russia_agg_rad, :dates)
    aggregate_df = DataFrames.combine(grouped, :aggregate_radiance => sum => :AggregateRadiance)
    sort!(aggregate_df, :dates)

    # only write once if you want
    # CSV.write(AGGREGATE_FILE, aggregate_df)
    println("Aggregation complete, DataFrame in memory.")
else
    println("No data processed, skipping aggregation.")
end

println("Script finished.")


In [ ]:
russia_agg_rad

##Ukraine

In [ ]:
ukraine_state_split = read_vector("Shedding-light-on-the-Russia-Ukraine-war/data/vector/ukraine.geojson")

In [ ]:
# ukraine_state_split = read_vector("/content/Shedding-light-on-the-Russia-Ukraine-war/data/vector/ukraine.geojson")

In [ ]:
const IGNORED_STATES = Set(["Donets'k", "Kherson", "Luhans'k", "Zaporizhia", "Crimea", "Sevastopol'"])
ukraine_state_split = filter(row -> !(row.NAME_1 in IGNORED_STATES), ukraine_state_split)

In [ ]:
radiance, cfobs = read_region_as_datacubes(
    ukraine_state_split.geometry[1];
    start_date=START_DATE,
    end_date=END_DATE,
    radiance_path=RADIANCE_RASTER_DIRECTORY,
    cf_path=CFOBS_RASTER_DIRECTORY
)

In [ ]:
radiance, cfobs = read_region_as_datacubes(ukraine_state_split[1, :].geometry;
    start_date=START_DATE, end_date=END_DATE,
    radiance_path=RADIANCE_RASTER_DIRECTORY,
    cf_path=CFOBS_RASTER_DIRECTORY)

In [ ]:
ukr_agg_rad = DataFrame()

for i in 1:nrow(ukraine_state_split)
    state_row = ukraine_state_split[i, :]
    state_name = state_row.NAME_1
    println("--- Processing state $i/$(nrow(ukraine_state_split)): $state_name ---")

    try
        state_df = process_state(state_row)
        if !isempty(state_df)
            append!(ukr_agg_rad, state_df; cols=:union)
        end
    catch e
        println("ERROR: Failed to process state '$state_name'. Skipping. Error: $e")
    end
    GC.gc()
end

println("--- Processing Complete ---")

# Final aggregation
if !isempty(ukr_agg_rad)
    println("Aggregating results...")
    grouped = groupby(ukr_agg_rad, :dates)
    aggregate_df = DataFrames.combine(grouped, :aggregate_radiance => sum => :AggregateRadiance)
    sort!(aggregate_df, :dates)

    # only write once if you want
    # CSV.write(AGGREGATE_FILE, aggregate_df)
    println("Aggregation complete, DataFrame in memory.")
else
    println("No data processed, skipping aggregation.")
end

println("Script finished.")


In [ ]:
ukr_agg_rad

##Disputed

In [ ]:
disp_state_split = read_vector("Shedding-light-on-the-Russia-Ukraine-war/data/vector/ukraine.geojson")

In [ ]:
# disp_state_split = read_vector("/content/Shedding-light-on-the-Russia-Ukraine-war/data/vector/ukraine.geojson")

In [ ]:
const IGNORED_STATES = Set(["Donets'k", "Kherson", "Luhans'k", "Zaporizhia", "Crimea", "Sevastopol'"])
disp_state_split = subset(disp_state_split, :NAME_1 => ByRow(x -> x in IGNORED_STATES))

In [ ]:
radiance, cfobs = read_region_as_datacubes(
    disp_state_split.geometry[1];
    start_date=START_DATE,
    end_date=END_DATE,
    radiance_path=RADIANCE_RASTER_DIRECTORY,
    cf_path=CFOBS_RASTER_DIRECTORY
)

In [ ]:
radiance, cfobs = read_region_as_datacubes(disp_state_split[1, :].geometry;
    start_date=START_DATE, end_date=END_DATE,
    radiance_path=RADIANCE_RASTER_DIRECTORY,
    cf_path=CFOBS_RASTER_DIRECTORY)

In [ ]:
disp_agg_rad = DataFrame()

for i in 1:nrow(disp_state_split)
    state_row = disp_state_split[i, :]
    state_name = state_row.NAME_1
    println("--- Processing state $i/$(nrow(disp_state_split)): $state_name ---")

    try
        state_df = process_state(state_row)
        if !isempty(state_df)
            append!(disp_agg_rad, state_df; cols=:union)
        end
    catch e
        println("ERROR: Failed to process state '$state_name'. Skipping. Error: $e")
    end
    GC.gc()
end

println("--- Processing Complete ---")

# Final aggregation
if !isempty(disp_agg_rad)
    println("Aggregating results...")
    grouped = groupby(disp_agg_rad, :dates)
    aggregate_df = DataFrames.combine(grouped, :aggregate_radiance => sum => :AggregateRadiance)
    sort!(aggregate_df, :dates)

    # only write once if you want
    # CSV.write(AGGREGATE_FILE, aggregate_df)
    println("Aggregation complete, DataFrame in memory.")
else
    println("No data processed, skipping aggregation.")
end

println("Script finished.")


In [ ]:
disp_agg_rad